d-sandbox

<div style="text-align: center; line-height: 0; padding-top: 9px;">
  <img src="https://databricks.com/wp-content/uploads/2018/03/db-academy-rgb-1200px.png" alt="Databricks Learning" style="width: 600px; height: 163px">
</div>

-sandbox
<img src="https://files.training.databricks.com/images/Apache-Spark-Logo_TM_200px.png" style="float: left: margin: 20px"/>

# Structured Streaming Concepts

## In this lesson you:
* Stream data from a file and write it out to a distributed file system
* List active streams
* Stop active streams

## Datasets Used
Data from 
`/mnt/training/definitive-guide/data/activity-data-stream.json/`
contains smartphone accelerometer samples from devices and users. 

The file consists of the following columns:

| Field          | Description |
| ------------- | ----------- |
| `Index` | unique identifier of event |
| `x` | acceleration in x-dir |
| `y` | acceleration in y-dir |
| `z` | acceleration in z-dir |
| `User` | unique user identifier |
| `Model` | i.e Nexus4 |
| `Device` | type of Model |
| `gt` | transportation mode |

<img alt="Side Note" title="Side Note" style="vertical-align: text-bottom; position: relative; height:1.75em; top:0.05em; transform:rotate(15deg)" src="https://files.training.databricks.com/static/images/icon-note.webp"/> The last column is named `gt`. This is an initialism for "ground truth". Wikipedia: <a href="https://en.wikipedia.org/wiki/Ground_truth" target="_blank">Ground Truth</a>

## CAUTION
* Streaming jobs will quickly exhaust your disk quota, if you are using Community Edition. If you find yourself running out of disk space, 
  you can [mount your own S3 bucket](https://docs.databricks.com/spark/latest/data-sources/aws/amazon-s3.html#mount-an-s3-bucket) and change
  your streaming paths accordingly.

<iframe  
src="//fast.wistia.net/embed/iframe/9649xyq1ae?videoFoam=true"
style="border:1px solid #1cb1c2;"
allowtransparency="true" scrolling="no" class="wistia_embed"
name="wistia_embed" allowfullscreen mozallowfullscreen webkitallowfullscreen
oallowfullscreen msallowfullscreen width="640" height="360" ></iframe>
<div>
<a target="_blank" href="https://fast.wistia.net/embed/iframe/9649xyq1ae?seo=false">
  <img alt="Opens in new tab" src="https://files.training.databricks.com/static/images/external-link-icon-16x16.png"/>&nbsp;Watch full-screen.</a>
</div>

<h2><img src="https://files.training.databricks.com/images/105/logo_spark_tiny.png"> Getting Started</h2>

Run the following cell to configure our "classroom."

In [5]:
%run "./Includes/Classroom-Setup"

<h2><img src="https://files.training.databricks.com/images/105/logo_spark_tiny.png"> Reading a Stream</h2>

The method `SparkSession.readStream` returns a `DataStreamReader` used to configure the stream.

There are a number of key points to the configuration of a `DataStreamReader`:
* The schema
* The type of stream: Files, Kafka, TCP/IP, etc
* Configuration specific to the type of stream
  * For files, the file type, the path to the files, max files, etc...
  * For TCP/IP the server's address, port number, etc...
  * For Kafka the server's address, port, topics, partitions, etc...

### The Schema

Every streaming DataFrame must have a schema - the definition of column names and data types.

Some sources such as Kafka define the schema for you.

In file-based streaming sources, for example, the schema is user-defined.

### Why must a schema be specified for a streaming DataFrame?

To say that another way... 

### Why are streaming DataFrames unable to infer/read a schema?

-sandbox

<script type="text/javascript">
  window.onload = function() {
    var allHints = document.getElementsByClassName("hint-8330");
    var answer = document.getElementById("answer-8330");
    var totalHints = allHints.length;
    var nextHint = 0;
    var hasAnswer = (answer != null);
    var items = new Array();
    var answerLabel = "Click here for the answer";
    for (var i = 0; i < totalHints; i++) {
      var elem = allHints[i];
      var label = "";
      if ((i + 1) == totalHints)
        label = answerLabel;
      else
        label = "Click here for the next hint";
      items.push({label: label, elem: elem});
    }
    if (hasAnswer) {
      items.push({label: '', elem: answer});
    }

    var button = document.getElementById("hint-button-8330");
    if (totalHints == 0) {
      button.innerHTML = answerLabel;
    }
    button.onclick = function() {
      items[nextHint].elem.style.display = 'block';
      if ((nextHint + 1) >= items.length)
        button.style.display = 'none';
      else
        button.innerHTML = items[nextHint].label;
        nextHint += 1;
    };
    button.ondblclick = function(e) {
      e.stopPropagation();
    }
    var answerCodeBlocks = document.getElementsByTagName("code");
    for (var i = 0; i < answerCodeBlocks.length; i++) {
      var elem = answerCodeBlocks[i];
      var parent = elem.parentNode;
      if (parent.name != "pre") {
        var newNode = document.createElement("pre");
        newNode.append(elem.cloneNode(true));
        elem.replaceWith(newNode);
        elem = newNode;
      }
      elem.ondblclick = function(e) {
        e.stopPropagation();
      };

      elem.style.marginTop = "1em";
    }
  };
</script>

<div>
  <button type="button" class="btn btn-light"
          style="margin-top: 1em"
          id="hint-button-8330">Click here for a hint</button>
</div>
<div id="answer-8330" style="padding-bottom: 20px; display: none">
  The answer:
  <div class="answer" style="margin-left: 1em">
If you have enough data, you can infer the schema.
&lt;br&gt;&lt;br&gt;
If you don&#x27;t have enough data you run the risk of miss-inferring the schema.
&lt;br&gt;&lt;br&gt;
For example, you think you have all integers but the last value contains &quot;1.123&quot; (a float) or &quot;snoopy&quot; (a string).
&lt;br&gt;&lt;br&gt;
With a stream, we have to assume we don&#x27;t have enough data because we are starting with zero records.
&lt;br&gt;&lt;br&gt;
And unlike reading from a table or parquet file, there is nowhere from which to &quot;read&quot; the stream&#x27;s schema.
&lt;br&gt;&lt;br&gt;
For this reason, we must specify the schema manually.
  </div>
</div>

In [10]:
# Define the schema using a DDL-formatted string.
dataSchema = "Recorded_At timestamp, Device string, Index long, Model string, User string, _corrupt_record String, gt string, x double, y double, z double"

In [11]:
# Another way to define schema
# Doc: https://spark.apache.org/docs/latest/api/python/pyspark.sql.html#pyspark.sql.types.StructType
# Code Detail: https://spark.apache.org/docs/latest/api/python/_modules/pyspark/sql/types.html#StructType.add

from pyspark.sql.types import StructType

dataSchema2 = StructType().add("Recorded_At", "timestamp").add("Device", "string").add("Index", "long").add("Model", "string").add("User", "string").add("_corrupt_record", "string").add("gt", "string").add("x", "double").add("y", "double").add("z", "double")



### Configuring a File Stream

In our example below, we will be consuming files written continuously to a pre-defined directory. 

To control how much data is pulled into Spark at once, we can specify the option `maxFilesPerTrigger`.

In our example below, we will be reading in only one file for every trigger interval:

`dsr.option("maxFilesPerTrigger", 1)`

Both the location and file type are specified with the following call, which itself returns a `DataFrame`:

`df = dsr.json(dataPath)`

In [13]:
dataPath = "dbfs:/mnt/training/definitive-guide/data/activity-data-stream.json"
initialDF = (spark
.readStream                           # Returns DataStreamReader
.option("maxFilesPerTrigger",1)       # Force processing of only 1 file per trigger 
.schema(dataSchema)                   # Required for all streaming DataFrames
.json(dataPath)                       # The stream's source directory and file type
)            






And with the initial `DataFrame`, we can apply some transformations:

In [15]:
streamingDF = (initialDF
.withColumnRenamed("Index","User_ID") # Pick a "better" column name
.drop("_corrupt_record") # Remove an unnecessary column



)

### Streaming DataFrames

Other than the call to `spark.readStream`, it looks just like any other `DataFrame`.

But is it a "streaming" `DataFrame`?

You can differentiate between a "static" and "streaming" `DataFrame` with the following call:

In [17]:
# Static vs Streaming?

streamingDF.isStreaming

Out[11]: True

### Unsupported Operations

Most operations on a "streaming" DataFrame are identical to a "static" DataFrame.

There are some exceptions to this.

One such example would be to sort our never-ending stream by `Recorded_At`.

In [19]:
from pyspark.sql.functions import col

try:
  sortedDF = streamingDF.orderBy(col("Recorded_At").desc())
  display(sortedDF)
except:
  print("Sorting is not supported on an unaggregated stream")

Sorting is not supported on an unaggregated stream

-sandbox
Sorting is one of a handful of operations that is either too complex or logically not possible to do with a stream.

For more information on this topic, see the <a href="https://spark.apache.org/docs/latest/structured-streaming-programming-guide.html#unsupported-operations" target="_blank">Structured Streaming Programming Guide / Unsupported Operations</a>.

<img alt="Side Note" title="Side Note" style="vertical-align: text-bottom; position: relative; height:1.75em; top:0.05em; transform:rotate(15deg)" src="https://files.training.databricks.com/static/images/icon-note.webp"/> We will see in the following module how we can sort an **aggregated** stream.

<iframe  
src="//fast.wistia.net/embed/iframe/agiu13vghf?videoFoam=true"
style="border:1px solid #1cb1c2;"
allowtransparency="true" scrolling="no" class="wistia_embed"
name="wistia_embed" allowfullscreen mozallowfullscreen webkitallowfullscreen
oallowfullscreen msallowfullscreen width="640" height="360" ></iframe>
<div>
<a target="_blank" href="https://fast.wistia.net/embed/iframe/agiu13vghf?seo=false">
  <img alt="Opens in new tab" src="https://files.training.databricks.com/static/images/external-link-icon-16x16.png"/>&nbsp;Watch full-screen.</a>
</div>

<h2><img src="https://files.training.databricks.com/images/105/logo_spark_tiny.png"> Writing a Stream</h2>

The method `DataFrame.writeStream` returns a `DataStreamWriter` used to configure the output of the stream.

There are a number of parameters to the `DataStreamWriter` configuration:
* Query's name (optional) - This name must be unique among all the currently active queries in the associated SQLContext.
* Trigger (optional) - Default value is `ProcessingTime(0`) and it will run the query as fast as possible.
* Checkpointing directory (optional)
* Output mode
* Output sink
* Configuration specific to the output sink, such as:
  * The host, port and topic of the receiving Kafka server
  * The file format and final destination of files
  * A custom sink via `dsw.foreach(...)`

Once the configuration is completed, we can trigger the job with a call to `dsw.start()`

### Triggers

The trigger specifies when the system should process the next set of data.

| Trigger Type                           | Example | Notes |
|----------------------------------------|-----------|-------------|
| Unspecified                            |  | _DEFAULT_- The query will be executed as soon as the system has completed processing the previous query |
| Fixed interval micro-batches           | `dsw.trigger(Trigger.ProcessingTime("6 hours"))` | The query will be executed in micro-batches and kicked off at the user-specified intervals |
| One-time micro-batch                   | `dsw.trigger(Trigger.Once())` | The query will execute _only one_ micro-batch to process all the available data and then stop on its own |
| Continuous w/fixed checkpoint interval | `dsw.trigger(Trigger.Continuous("1 second"))` | The query will be executed in a low-latency, continuous processing mode. _EXPERIMENTAL_ in 2.3.2 |

In the example below, you will be using a fixed interval of 3 seconds:

`dsw.trigger(Trigger.ProcessingTime("3 seconds"))`

### Checkpointing

A <b>checkpoint</b> stores the current state of your streaming job to a reliable storage system such as Amazon S3or HDFS. It does not store the state of your streaming job to the local file system of any node in your cluster. 

Together with write ahead logs, a terminated stream can be restarted and it will continue from where it left off.

To enable this feature, you only need to specify the location of a checkpoint directory:

`dsw.option("checkpointLocation", checkpointPath)`

Points to consider:
* If you do not have a checkpoint directory, when the streaming job stops, you lose all state around your streaming job and upon restart, you start from scratch.
* For some sinks, you will get an error if you do not specify a checkpoint directory:<br/>
`analysisException: 'checkpointLocation must be specified either through option("checkpointLocation", ...)..`
* Also note that every streaming job should have its own checkpoint directory: no sharing.

### Output Modes

| Mode   | Example | Notes |
| ------------- | ----------- |
| **Complete** | `dsw.outputMode("complete")` | The entire updated Result Table is written to the sink. The individual sink implementation decides how to handle writing the entire table. |
| **Append** | `dsw.outputMode("append")`     | Only the new rows appended to the Result Table since the last trigger are written to the sink. |
| **Update** | `dsw.outputMode("update")`     | Only the rows in the Result Table that were updated since the last trigger will be outputted to the sink. Since Spark 2.1.1 |

In the example below, we are writing to a Parquet directory which only supports the `append` mode: 

`dsw.outputMode("append")`

### Output Sinks

`DataStreamWriter.format` accepts the following values, among others:

| Output Sink | Example                                          | Notes |
| ----------- | ------------------------------------------------ | ----- |
| **File**    | `dsw.format("parquet")`, `dsw.format("csv")`...  | Dumps the Result Table to a file. Supports Parquet, json, csv, etc.|
| **Kafka**   | `dsw.format("kafka")`      | Writes the output to one or more topics in Kafka |
| **Console** | `dsw.format("console")`    | Prints data to the console (useful for debugging) |
| **Memory**  | `dsw.format("memory")`     | Updates an in-memory table, which can be queried through Spark SQL or the DataFrame API |
| **foreach** | `dsw.foreach(writer: ForeachWriter)` | This is your "escape hatch", allowing you to write your own type of sink. |
| **Delta**    | `dsw.format("delta")`     | A proprietary sink |

In the example below, we will be appending files to a Parquet file and specifying its location with this call:

`dsw.format("parquet").start(outputPathDir)`

<h2><img src="https://files.training.databricks.com/images/105/logo_spark_tiny.png"> Let's Do Some Streaming</h2>

In the cell below, we write data from a streaming query to `outputPathDir`. 

There are a couple of things to note below:
0. We are giving the query a name via the call to `queryName` . We can use this later to reference the query by name.
0. Spark begins running jobs once we call `start`. This is equivilent to calling an action on a "static" DataFrame.
0. The call to `start` returns a `StreamingQuery` object. We can use this to interact with the running query.

In [29]:
basePath = userhome + "/structured-streaming-concepts/python" # A working directory for our streaming app
dbutils.fs.mkdirs(basePath)                                   # Make sure that our working directory exists
outputPathDir = basePath + "/output.parquet"                  # A subdirectory for our output
checkpointPath = basePath + "/checkpoint"                     # A subdirectory for our checkpoint & W-A logs

streamingQuery = (streamingDF # Start with our "streaming" DataFrame
.writeStream # Get the DataStreamWriter
.queryName("stream_1p")# Name the query
.trigger(processingTime="3 seconds")# Configure for a 3-second micro-batch
.format("parquet")               # Specify the sink type, a Parquet file
.option("checkpointLocation",checkpointPath)  # Specify the location of checkpoint files & W-A logs
.outputMode("append")    # Write only new data to the "file"                 
.start(outputPathDir)    # Start the job, writing to the specified directory


)

In [30]:
# Wait until stream is done initializing...
untilStreamIsReady("stream_1p")

The stream is active and ready.

<h2><img src="https://files.training.databricks.com/images/105/logo_spark_tiny.png"> Managing Streaming Queries</h2>

When a query is started, the `StreamingQuery` object can be used to monitor and manage the query.

| Method    |  Description |
| ----------- | ------------------------------- |
|`id`| get unique identifier of the running query that persists across restarts from checkpoint data |
|`runId`| get unique id of this run of the query, which will be generated at every start/restart |
|`name`| get name of the auto-generated or user-specified name |
|`explain()`| print detailed explanations of the query |
|`stop()`| stop query |
|`awaitTermination()`| block until query is terminated, with stop() or with error |
|`exception`| exception if query terminated with error |
|`recentProgress`| array of most recent progress updates for this query |
|`lastProgress`| most recent progress update of this streaming query |

In [32]:
streamingQuery.recentProgress

Out[15]: [{'id': '9c90a2af-92ab-4116-9d7a-285fb5da1af7',
 'runId': 'bb9022ff-b4cc-447e-98fe-b59010848f74',
 'name': 'stream_1p',
 'timestamp': '2020-02-26T18:22:12.878Z',
 'batchId': 0,
 'numInputRows': 80096,
 'processedRowsPerSecond': 8786.309784993418,
 'durationMs': {'addBatch': 6804,
 'getBatch': 135,
 'getOffset': 954,
 'queryPlanning': 114,
 'triggerExecution': 9099,
 'walCommit': 462},
 'stateOperators': [],
 'sources': [{'description': 'FileStreamSource[dbfs:/mnt/training/definitive-guide/data/activity-data-stream.json]',
 'startOffset': None,
 'endOffset': {'logOffset': 0},
 'numInputRows': 80096,
 'processedRowsPerSecond': 8786.309784993418}],
 'sink': {'description': 'FileSink[dbfs:/user/shashank.rao@rhsmith.umd.edu/structured-streaming-concepts/python/output.parquet]'}},
 {'id': '9c90a2af-92ab-4116-9d7a-285fb5da1af7',
 'runId': 'bb9022ff-b4cc-447e-98fe-b59010848f74',
 'name': 'stream_1p',
 'timestamp': '2020-02-26T18:22:22.035Z',
 'batchId': 1,
 'numInputRows': 68545,
 'inputRowsPerSecond': 7485.530195478868,
 'processedRowsPerSecond': 15424.167416741675,
 'durationMs': {'addBatch': 2537,
 'getBatch': 106,
 'getOffset': 674,
 'queryPlanning': 27,
 'triggerExecution': 4444,
 'walCommit': 706},
 'stateOperators': [],
 'sources': [{'description': 'FileStreamSource[dbfs:/mnt/training/definitive-guide/data/activity-data-stream.json]',
 'startOffset': {'logOffset': 0},
 'endOffset': {'logOffset': 1},
 'numInputRows': 68545,
 'inputRowsPerSecond': 7485.530195478868,
 'processedRowsPerSecond': 15424.167416741675}],
 'sink': {'description': 'FileSink[dbfs:/user/shashank.rao@rhsmith.umd.edu/structured-streaming-concepts/python/output.parquet]'}},
 {'id': '9c90a2af-92ab-4116-9d7a-285fb5da1af7',
 'runId': 'bb9022ff-b4cc-447e-98fe-b59010848f74',
 'name': 'stream_1p',
 'timestamp': '2020-02-26T18:22:26.481Z',
 'batchId': 2,
 'numInputRows': 71211,
 'inputRowsPerSecond': 16016.869095816464,
 'processedRowsPerSecond': 20369.279176201373,
 'durationMs': {'addBatch': 2178,
 'getBatch': 66,
 'getOffset': 466,
 'queryPlanning': 18,
 'triggerExecution': 3495,
 'walCommit': 375},
 'stateOperators': [],
 'sources': [{'description': 'FileStreamSource[dbfs:/mnt/training/definitive-guide/data/activity-data-stream.json]',
 'startOffset': {'logOffset': 1},
 'endOffset': {'logOffset': 2},
 'numInputRows': 71211,
 'inputRowsPerSecond': 16016.869095816464,
 'processedRowsPerSecond': 20369.279176201373}],
 'sink': {'description': 'FileSink[dbfs:/user/shashank.rao@rhsmith.umd.edu/structured-streaming-concepts/python/output.parquet]'}},
 {'id': '9c90a2af-92ab-4116-9d7a-285fb5da1af7',
 'runId': 'bb9022ff-b4cc-447e-98fe-b59010848f74',
 'name': 'stream_1p',
 'timestamp': '2020-02-26T18:22:29.978Z',
 'batchId': 3,
 'numInputRows': 81992,
 'inputRowsPerSecond': 23446.38261366886,
 'processedRowsPerSecond': 18689.765215409167,
 'durationMs': {'addBatch': 2409,
 'getBatch': 68,
 'getOffset': 497,
 'queryPlanning': 19,
 'triggerExecution': 4387,
 'walCommit': 964},
 'stateOperators': [],
 'sources': [{'description': 'FileStreamSource[dbfs:/mnt/training/definitive-guide/data/activity-data-stream.json]',
 'startOffset': {'logOffset': 2},
 'endOffset': {'logOffset': 3},
 'numInputRows': 81992,
 'inputRowsPerSecond': 23446.38261366886,
 'processedRowsPerSecond': 18689.765215409167}],
 'sink': {'description': 'FileSink[dbfs:/user/shashank.rao@rhsmith.umd.edu/structured-streaming-concepts/python/output.parquet]'}},
 {'id': '9c90a2af-92ab-4116-9d7a-285fb5da1af7',
 'runId': 'bb9022ff-b4cc-447e-98fe-b59010848f74',
 'name': 'stream_1p',
 'timestamp': '2020-02-26T18:22:34.366Z',
 'batchId': 4,
 'numInputRows': 73878,
 'inputRowsPerSecond': 16836.37192342753,
 'processedRowsPerSecond': 22571.952337305225,
 'durationMs': {'addBatch': 1821,
 'getBatch': 122,
 'getOffset': 485,
 'queryPlanning': 34,
 'triggerExecution': 3273,
 'walCommit': 430},
 'stateOperators': [],
 'sources': [{'description': 'FileStreamSource[dbfs:/mnt/training/definitive-guide/data/activity-data-s

Additionally, we can iterate over a list of active streams:

In [34]:
for s in spark.streams.active:         # Iterate over all streams
   print("{}:  {}".format(s.id,s.name)) #Print the stream's id and name

9c90a2af-92ab-4116-9d7a-285fb5da1af7: stream_1p

The code below stops the `streamingQuery` defined above and introduces `awaitTermination()`

`awaitTermination()` will block the current thread
* Until the stream stops or 
* Until the specified timeout elapses

In [36]:
streamingQuery.awaitTermination(5)      # Stream for another 5 seconds while the current thread blocks
streamingQuery.stop()                   # Stop the stream

-sandbox

<h2><img src="https://files.training.databricks.com/images/105/logo_spark_tiny.png"> The Display function</h2>

Within the Databricks notebooks, we can use the `display()` function to render a live plot

#### One Gotcha!
When you pass a "streaming" `DataFrame` to `display`:
* A "memory" sink is being used
* The output mode is complete
* The query name is specified with the `streamName` parameter
* The trigger is specified with the `trigger` parameter
* The checkpointing location is specified with the `checkpointLocation`

`display(myDF, streamName = "myQuery")`

<img alt="Side Note" title="Side Note" style="vertical-align: text-bottom; position: relative; height:1.75em; top:0.05em; transform:rotate(15deg)" src="https://files.training.databricks.com/static/images/icon-note.webp"/> We just programmatically stopped our only streaming query in the previous cell. In the cell below, `display` will automatically start our streaming DataFrame, `streamingDF`.  We are passing `stream_2p` as the name for this newly started stream.

In [38]:
myStream = "stream_2p"
display(streamingDF,streamName=myStream)

Recorded_At,Device,User_ID,Model,User,gt,x,y,z
2015-02-23T10:18:54.958+0000,nexus4_2,0,nexus4,g,stand,-3.814697E-4,0.03656006,0.030136108
2015-02-23T10:18:54.964+0000,nexus4_2,1,nexus4,g,stand,-0.001449585,0.035491943,0.027999878
2015-02-23T10:18:54.968+0000,nexus4_2,2,nexus4,g,stand,6.866455E-4,0.033355713,0.030136108
2015-02-23T10:18:54.976+0000,nexus4_2,3,nexus4,g,stand,6.866455E-4,0.033355713,0.022659302
2015-02-23T10:18:54.978+0000,nexus4_2,4,nexus4,g,stand,-3.814697E-4,0.030151367,0.025863647
2015-02-23T10:18:54.992+0000,nexus4_2,5,nexus4,g,stand,-3.814697E-4,0.025878906,0.023727417
2015-02-23T10:18:54.992+0000,nexus4_2,6,nexus4,g,stand,-0.001449585,0.02267456,0.023727417
2015-02-23T10:18:54.996+0000,nexus4_2,7,nexus4,g,stand,-3.814697E-4,0.019470215,0.024795532
2015-02-23T10:18:54.999+0000,nexus4_2,8,nexus4,g,stand,6.866455E-4,0.01626587,0.021591187
2015-02-23T10:18:55.000+0000,nexus4_1,0,nexus4,g,stand,0.80996704,-0.34129333,-0.1672821


In [39]:
# Wait until stream is done initializing...
untilStreamIsReady(myStream)

The stream is active and ready.

Using the value passed to `streamName` in the call to `display`, we can programatically access this specific stream:

In [41]:
print("Looking for {}".format(myStream))

for stream in spark.streams.active:      # Loop over all active streams
  if stream.name == myStream:            # Single out "streamWithTimestamp"
    print("Found {} ({})".format(stream.name, stream.id)) 

Looking for stream_2p
Found stream_2p (d660790c-0e35-434a-a61a-73d70574c192)

Stop all remaining streams.

In [43]:
for s in spark.streams.active:
  s.stop()

Clean up our directories

In [45]:
dbutils.fs.rm(basePath, True)

Out[24]: True

## End-to-end Fault Tolerance

Structured Streaming ensures end-to-end exactly-once fault-tolerance guarantees through _checkpointing_ and <a href="https://en.wikipedia.org/wiki/Write-ahead_logging" target="_blank">Write Ahead Logs</a>.

Structured Streaming sources, sinks, and the underlying execution engine work together to track the progress of stream processing. If a failure occurs, the streaming engine attempts to restart and/or reprocess the data.

This approach _only_ works if the streaming source is replayable. To ensure fault-tolerance, Structured Streaming assumes that every streaming source has offsets, akin to:

* <a target="_blank" href="https://kafka.apache.org/documentation/#intro_topics">Kafka message offsets</a>
* <a target="_blank" href="http://docs.aws.amazon.com/streams/latest/dev/key-concepts.html#sequence-number">Kinesis sequence numbers</a>

At a high level, the underlying streaming mechanism relies on a couple approaches:

* First, Structured Streaming uses checkpointing and write-ahead logs to record the offset range of data being processed during each trigger interval.
* Next, the streaming sinks are designed to be _idempotent_—that is, multiple writes of the same data (as identified by the offset) do _not_ result in duplicates being written to the sink.

Taken together, replayable data sources and idempotent sinks allow Structured Streaming to ensure **end-to-end, exactly-once semantics** under any failure condition.

<h2><img src="https://files.training.databricks.com/images/105/logo_spark_tiny.png"> Summary</h2>

We use `readStream` to read streaming input from a variety of input sources and create a DataFrame.

Nothing happens until we invoke `writeStream` or `display`. 

Using `writeStream` we can write to a variety of output sinks. Using `display` we draw LIVE bar graphs, charts and other plot types in the notebook.

<h2><img src="https://files.training.databricks.com/images/105/logo_spark_tiny.png"> Review Questions</h2>

**Q:** What do `readStream` and `writeStream` do?<br>
**A:** `readStream` creates a streaming DataFrame.<br>`writeStream` sends streaming data to a directory or other type of output sink.

**Q:** What does `display` output if it is applied to a DataFrame created via `readStream`?<br>
**A:** `display` sends streaming data to a LIVE graph!

**Q:** When you do a write stream command, what does this option do `outputMode("append")` ?<br>
**A:** This option takes on the following values and their respective meanings:
* <b>append</b>: add only new records to output sink
* <b>complete</b>: rewrite full output - applicable to aggregations operations
* <b>update</b>: update changed records in place

**Q:** What happens if you do not specify `option("checkpointLocation", pointer-to-checkpoint directory)`?<br>
**A:** When the streaming job stops, you lose all state around your streaming job and upon restart, you start from scratch.

**Q:** How do you view the list of active streams?<br>
**A:** Invoke `spark.streams.active`.

**Q:** How do you verify whether `streamingQuery` is running (boolean output)?<br>
**A:** Invoke `spark.streams.get(streamingQuery.id).isActive`.

<h2><img src="https://files.training.databricks.com/images/105/logo_spark_tiny.png"> Next Steps</h2>

Start the next lab, [Streaming Concepts Lab]($./Labs/SS 02 - Streaming Concepts Lab).

<h2><img src="https://files.training.databricks.com/images/105/logo_spark_tiny.png"> Additional Topics &amp; Resources</h2>
* <a href="https://spark.apache.org/docs/latest/structured-streaming-programming-guide.html#" target="_blank">Structured Streaming Programming Guide</a>
* <a href="https://www.youtube.com/watch?v=rl8dIzTpxrI" target="_blank">A Deep Dive into Structured Streaming</a> by Tathagata Das. This is an excellent video describing how Structured Streaming works.

-sandbox
&copy; 2019 Databricks, Inc. All rights reserved.<br/>
Apache, Apache Spark, Spark and the Spark logo are trademarks of the <a href="http://www.apache.org/">Apache Software Foundation</a>.<br/>
<br/>
<a href="https://databricks.com/privacy-policy">Privacy Policy</a> | <a href="https://databricks.com/terms-of-use">Terms of Use</a> | <a href="http://help.databricks.com/">Support</a>